# Módulo de preprocesamiento 
Ammi Beltrán & Fernanda Borja

## Importado de librerias

In [ ]:
# ! pip install -U scikit-learn

In [105]:
import os
import mne
import glob
#
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import pandas as pd

# import torch 

ModuleNotFoundError: No module named 'torch'

Se crea el path de data

In [50]:
if not os.path.exists("data"):
    os.makedirs("data")
    print("Data directory created :D")

Data directory created :D


Dirección del dataset

In [3]:
# EDFDIR = "D:\\OneDrive\\OneDrive - Universidad de Chile\\Semestre X\\Inteligencia\\Proyecto\\dataset\\tuh_eeg"
EDFDIR = "c:\\Users\\TheSy\\Desktop\\tuh_eeg"
# files = glob.glob(EDFDIR + '/**/*.edf', recursive=True)

## Preprocesamiento

In [ ]:
# Usamos MNE
data = mne.io.read_raw_edf(files[0])
raw_data = data.get_data()
info = data.info
channels = data.ch_names

### Funciones
* Una vez funcione mover a .py y traer como librería 

Seleccion de canales

In [4]:
def channel_select(data, channels):
    '''
    Selects channels from array 
    '''
    extracted = data.pick(channels, exclude="bads")
    return extracted

Clipeo

In [5]:
def clip(data, channels,max= 500e-6):
    def cliper(array):
        for i in range(len(array)):
            if abs(array[i]) > max:
                array[i] = math.copysign(max,array[i])
        return array
    data.apply_function(cliper, picks=channels, channel_wise= True)

Filtrado

In [6]:
def eeg_filter(data, lfreq = 0.3, hfreq= 80):
    '''
    
    '''
    data_copy = copy.copy(data)
    filtered = data_copy.filter(l_freq = lfreq,
                                h_freq = hfreq,
                                method = "iir"
                                )
    return filtered

Corte de primero minuto y de max

In [7]:
def temporal_crop(data, tin = 60, tfin = 12*60):
    ''' 
    Cut the channels from the second "tin" to "tfin"
    '''
    data_copy = copy.copy(data)
    croped = data_copy.crop(tmin = tin, tmax = min(tfin, int(data.times[-1])))
    return croped

Marcado de épocas

In [ ]:
# eventos
def get_epochs(data, channels, window = 20.0):
    ''' 
    window es la ventana de tiempo
    '''
    data_copy = copy.copy(data)
    # Create events
    events = mne.make_fixed_length_events(data_copy, duration = window, first_samp = True)
    # Divide accordingly
    picks = channels
    epochs = mne.Epochs(raw = data_copy, events = events, picks = picks, preload = True,
                        tmin = 0., tmax = window, baseline = None,
                        flat = dict(eeg = 1e-6))
    
    epochs.drop(-1,reason = "Unfixed duration")
    return epochs

Downsample

In [8]:
def downsample(epoch, freq = 200):
    ''' 
    Downsamples the data given by a factor
    En nuestro caso, down corresponde a (frecuencia que queremos)/(frecuencia actual)
    '''
    down = epoch.resample(freq, npad = "auto")
    return down

Normalización

In [9]:
def normalization(epochs):
    obj = mne.decoding.Scaler(info = epochs.info, scalings='mean')
    values = obj.fit_transform(epochs.get_data())
    return values

***

Union

In [ ]:
def preprocessing(path):
    ''' 
    Pipeline de procesamiento c:
    - Canales c:
    - Filtrado c:
    - Downsample c:
    - Crop temporal (primer minuto y a los 11 minutos) c:  
    - Segmentación c:
    - Normalización c:
    '''

    files = glob.glob(path + '/**/*.edf', recursive = True)
    # windows = np.empty() EVALUAR
    windows = []
    for i in range(0, len(files)):
        data = mne.io.read_raw_edf(files[i], preload=True)
        # raw_data = data.get_data()
        # info = data.info
        channels = data.ch_names[:21]
        # Pipeline
        # 
        # mapping = rename_channels(data)

        # data.rename_channels(data.info, mapping)
        # data.rename_channels(mapping)

        # try:
        channel_data = channel_select(data, channels)
        clip(channel_data,channels)
        # except:
        #     channel_data = channel_select(data, CHANNELS_LE)
        #     ch = CHANNELS_LE
        if(int(data.times[-1]) < 100):
            continue
        filtered = eeg_filter(channel_data)
        trimmed_data = temporal_crop(filtered)
        epochs = get_epochs(trimmed_data, channels)
        down_data = downsample(epochs)
        norm_data = normalization(down_data)
        #
        windows.append(norm_data)
    return windows

In [ ]:
# patient = glob.glob(EDFDIR + '/**')
# patient

## Uso de funciones

In [ ]:

win = preprocessing(EDFDIR)
win

***

In [ ]:
# Channels we want

def rename_channels(data):
    # info = data.info
    original = data.ch_names
    new = copy.copy(original)
    res = {}
    for ch in new:
        
        isin = False
        for n in CHANNELS:
            if n in ch:
                res[ch] = n
                isin = True
                break

        if not isin:
            res[ch] = ch
            
    return res


In [98]:
def EDFprep(edf):
    '''
    Pipeline
    '''
    pass

In [106]:
def prep(path, save = False, save_dir = "data", sep = "\\"):
    ''' 
    Lectura de todos los edfs de cada paciente, guardado de ventanas temporales y csv de direcciones

    Inputs:
        -path
    Output:
        -dir_csv: csv con todos los datos de guardado de las ventanas de cada edf.    
    '''
    LEN_PAT = 8
    SESION_LEN = 15
    loc_df = pd.DataFrame(columns= ["Patient", "Session","N_Win", "Dir"], )

    if save:
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            print("Data directory created :D")
    

    patient_path = glob.glob(EDFDIR + '/**')
    for patient in patient_path:

        #Para guardar la id en el DF
        patient_id = patient[-LEN_PAT:]
        
        sessions = glob.glob(patient + '/**')
        for session in sessions:

                    
            #Para guardar la sesion correspondiente
            session_id = session[-SESION_LEN:-(SESION_LEN - 4)]

            if save:
                if not os.path.exists(save_dir + sep + session_id):
                    os.makedirs(save_dir + sep + session_id)

            edfs = glob.glob(session+ "/**/*.edf")
            for edf in edfs:

                raw = mne.io.read_raw_edf(edf,preload=True)
                data = EDFprep(raw)
                
                for i in range(len(data)):
                    sdir = f"{save_dir}{sep}{patient_id}{sep}{session_id}{sep}w{i+1}.pt"
                    loc_df.loc[len(loc_df)] = [patient_id,session_id,i+1,sdir]

                    if save:
                        torch.save(data[i],f"{save_dir}{sep}{session}{sep}{patient_id}_{session}_w{i+1}.pt")

    if save:
        pass
        #Guardar df como csv

                
    return loc_df
            
    ["aaaa","s001",7,"data/s001/aaaa_s001_w7.pt"]

    win = [[[]]]
        
        
    win = torch.load("w7.pt")

    Batch = [].cat(win)

    # windows = []
    # for i in range(0, len(files)):
    #     data = mne.io.read_raw_edf(files[i], preload=True)

    #     channels = data.ch_names[:21]
      
    #     channel_data = channel_select(data, channels)

    #     clip(channel_data,channels)

    #     if(int(data.times[-1]) < 100):
    #         continue

    #     filtered = eeg_filter(channel_data)
    #     trimmed_data = temporal_crop(filtered)
    #     epochs = get_epochs(trimmed_data, channels)
    #     down_data = downsample(epochs)
    #     norm_data = normalization(down_data)
        
    #     windows.append(norm_data)
    # return windows

In [97]:
prep(EDFDIR)

s001
s001
s002
s003
s001
s002
s003
s004
s005
s006
s007
s008
s009
s001
s001
s001
s002
s003
s004
s005
s001
s002
s003
s004
s005
s006
s001
s002
s003
s004
s001
s001
s001
s001
s001
s001
s001
s002
s003
s004
s001
s002
s003
s001
s002
s003
s004
s005
s006
s007
s009
s001
s002
s003
s001
s002
s001
s002
s001
s001
s002
s003
s004
s005
s001
s002
s001
s002
s001
s001
s001
s001
s002
s001
s001
s001
s001
s001
s002
s003
s001
s001
s001
s002
s001
s001
s002
s003
s001
s002
s003
s001
s002
s003
s004
s005
s001
s001
s001
s001
s001
s001
s002
s001
s001
s001
s001
s002
s003
s001
s001
s001
s001
s002
s003
s004
s001
s002
s003
s004
s001
s001
s001
s001
s002
s003
s001
s002
s001
s002
s003
s001
s001
s001
s001
s002
s001
s001
s001
s001
s002
s003
s004
s005
s006
s007
s008
s009
s010
s011
s012
s013
s014
s015
s016
s017
s018
s001
s002
s003
s001
s001
s001
s001
s001
s001
s002
s001
s001
s003
s004
s005
s006
s007
s008
s001
s002
s003
s004
s001
s001
s001
s001
s001
s002
s001
s002
s001
s001
s002
s003
s004
s005
s006
s007
s008
s009
s010
s001
s002


In [48]:
dir_csv = pd.DataFrame(columns= ["Patient", "Nro_Set", "N_samp", "Dir"], )
# p1 = {"Patient": "aaaa", "Nro_Set": "s000", "N_samp": 1, "Dir": "c:\\UwU"}
dir_csv.loc[len(dir_csv)] = ["aaaa","s000",1,"c:\\UwU"]
# dir_csv.loc[1] = ["aaab","s000",1,"c:\\UwU"]

In [49]:
dir_csv

,Patient,Nro_Set,N_samp,Dir
0,aaaa,s000,1,c:\UwU


In [46]:
dir_csv.loc[len(dir_csv)] = ["aaaa","s000",1,"c:\\UwU"]